<a href="https://colab.research.google.com/github/ArunMunagala7/Zenskar-Assignment/blob/main/Zenskar_Assignment_Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading required Python Packages

In [1]:
!pip install --upgrade google-cloud-aiplatform
!pip install google-cloud-aiplatform
!pip install pymupdf
!pip install pypdf2
!pip install requests
!gcloud auth application-default login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 8.4 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.74.0
    Uninstalling google-cloud-aiplatform-1.74.0:
      Successfully uninstalled google-cloud-aiplatform-1.74.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=1i1y5GMCGn8oqcXmzilZVEHEVm2aNh&prompt=consent&token_usage=remote&access_type=offline&code_challenge=2cU5VOgrf015wXIhqCf-Zs1PDEM3-R-LskQzAdNVejE&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRrunpLPpOpAt40YQmWbnh1OGw-Q2WwLACfNYlbPi3iAXl70wfytoHzdmUJ_YWlP_HA

Credentials saved to file:

Importing necessary libraries for text extraction



In [2]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

#**1. Using PyPDF to extract the contents of the contract PDF**

This code extracts text from a legal contract PDF, uses Google's Vertex AI Gemini model to structure it into a clear, organized format with headings and detailed field values, and prints the formatted output for easy readability

In [3]:
import vertexai
from vertexai.generative_models import GenerativeModel
from PyPDF2 import PdfReader

# Initialize Vertex AI
vertexai.init(project="zenskar-assignment-445217", location="us-central1")

# Load the Gemini model
model = GenerativeModel("gemini-1.5-flash-002")

# System prompt for extraction
prompt = """
//You are given the contents a Legal Contract Document

Extract the contents of this document in a structured and organized manner. Ensure every field is followed by its respective value in a well organized manner with clear headings and subheadings as and when necessary.

Whenever tabular data is present, describe the data clearly and extract it mentioning in orderthe respective columns/rows and so on

IMPORTANT: Ensure every field in the contract is extracted along with its value IN SEPARATE LINES to ensure better readability WITHOUT FAIL
"""

# Path to your PDF file
pdf_file_path = "/content/sample contract.pdf"

# Extract text from the PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_content = extract_text_from_pdf(pdf_file_path)

# Combine the extracted text and the prompt
contents = [f"{prompt}\n\n{pdf_content}"]

# Generate content using Gemini
response = model.generate_content(contents)

contract_text=response.text
# Print the structured response
print(response.text)


/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


**Order Form**

**Date:** December 13, 2023
**Expires in:** 30 days

**Company:** Company A
**Address:** 9th Floor, Boston, MA 02111

**Client Information**

**Client:** Company B
**Client Contact Name:** Rob Keatts
**Client Contact Email:** abc@companyb.comn
**Billing Contact:**  (Not specified)
**Bill to Email:** (Not specified)
**Bill to Address:** (Not specified)


**Services Billing:** Subscription Billing
**Payment Terms:** Net 30

**Products - Professional Services**

| Product Services | Description                                                                                                                     | Price     | Discount |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------|-----------|----------|
| Product A         | Per Statement of Work SOW-CFCU-WCOBCMC-2023                                                                                 | $10,000   | $0       |

#**2. Automated Contract Field Extraction and Validation***

This code extracts key fields from a contract using Vertex AI's Gemini model, validates them with regex, and outputs structured JSON for both the fields and their reasoning logs.

In [10]:
import json
import re

# Step 1: Generate the dynamic prompt for the LLM
def create_dynamic_prompt_with_reasoning(fields, contract_text):
    """
    Generate a dynamic prompt for the LLM to extract fields along with reasoning.
    """
    step_1_instructions = "1. Extract the fields exactly as they appear in the text, preserving the raw format from the contract. Use the following keys:"
    step_1_keys = "\n".join([f"- \"{field}\"" for field in fields.keys()])

    step_2_instructions = (
        "2. For each field extracted, provide a reasoning log that explains why and where the field was extracted from the text.MAKE IT DETAILED AND ELABORATE"
    )
    step_2_example = """
    Example Output:
    {
        "extracted_fields": {
            "Contract ID": "123-ABC",
            "Customer Name": "Reynolds Consumer Products Inc.",
            "Contract Start Date": "2020-02-04",
            "Contract End Date": "2021-02-04",
            "Payment Terms": "Net 30",
            "Contract Amount": "$10,000.00",
            "Billing Frequency": "Monthly",
            "Contract Type": "Subscription"
        },
        "reasoning_log": {
            "Contract ID": "Detected near the 'Agreement Number' label in the document.",
            "Customer Name": "Extracted from the 'Client' section of the text.",
            "Contract Start Date": "Identified using keywords like 'Effective Date'.",
            "Contract End Date": "Identified using keywords like 'Expiration Date'.",
            "Payment Terms": "Matched payment terms format like 'Net X'.",
            "Contract Amount": "Extracted near the 'Total Amount' section.",
            "Billing Frequency": "Detected as a recurring pattern in the document.",
            "Contract Type": "Matched with keywords like 'Agreement Type'."
        }
    }
    """

    final_output_instruction = "Provide only the final JSON output containing both 'extracted_fields' and 'reasoning_log'. Do not include any intermediate steps, explanations, or raw iterations."

    # Assemble the full prompt
    prompt = f"""
    Extract the following fields from the contract text and generate a reasoning log:

    {step_1_instructions}
    {step_1_keys}

    {step_2_instructions}
    {step_2_example}

    {final_output_instruction}
    Contract Text:
    {contract_text}
    """
    return prompt.strip()


# Step 2: Define the fields and guidelines
fields = {
    "Contract ID": "Must be alphanumeric with optional hyphens (e.g., 'ABC-123'). If missing, look for unique identifiers labeled as 'Contract ID' or near terms like 'Agreement Number' or 'Reference Number'.",
    "Customer Name": "Must be a valid name as a string. If missing, search for names associated with 'Customer', 'Client', or 'Party'.",
    "Contract Start Date": "Must follow the format 'YYYY-MM-DD'. If missing, infer from phrases like 'Effective Date', 'Start Date', or 'Commencement Date'.",
    "Contract End Date": "Must follow the format 'YYYY-MM-DD'. If missing, infer from phrases like 'End Date', 'Expiration Date', or 'Termination Date'.",
    "Payment Terms": "Must represent terms of payment, which can include phrases like 'Net X' (e.g., 'Net 30'), 'Due in X days', or any other description indicating when payment is expected.",
    "Contract Amount": "Must be a valid currency format (e.g., '$10,000.00'). If missing, look for numeric values near 'Amount', 'Fee', or 'Total'.",
    "Billing Frequency": "Must indicate how often payments or services are recurring. This can include terms like 'Monthly', 'Yearly', 'Weekly', 'Daily', or phrases like 'every month', 'annually', 'per week', etc.",
    "Contract Type": "Must indicate the type of contract, such as 'Subscription', 'Fixed', 'Hourly', or 'Project-Based'. This can include phrases like 'Subscription Agreement', 'One-time Fixed Fee', 'Hourly Rate', 'Project Work', or similar."
}


# Step 3: Generate LLM Output Using Gemini
def generate_llm_output_with_reasoning(prompt):
    """
    Generate output from Gemini using the dynamic prompt, including reasoning.
    """
    # Initialize Vertex AI Gemini model
    model = GenerativeModel("gemini-1.5-pro-002")

    # Generate content using the prompt
    response = model.generate_content(prompt)

    # Extract the text content from the response
    llm_output = response.text

    # Clean the output to ensure it contains only valid JSON
    try:
        # Find the JSON object in the response using regex
        match = re.search(r"\{.*\}", llm_output, re.DOTALL)
        if match:
            return json.loads(match.group(0))
        else:
            raise ValueError("No valid JSON found in the LLM output.")
    except Exception as e:
        raise ValueError(f"Error extracting JSON from LLM output: {e}")


# Step 4: Refine Extracted Fields Using Regex Without Altering Reasoning Log
def refine_fields_with_regex_only(llm_output):
    """
    Refine and validate the LLM output using regex.
    """
    extracted_fields = llm_output.get("extracted_fields", {})

    # Define regex patterns for each field
    patterns = {
        "Contract ID": r"^[A-Za-z0-9\-]+$",  # Alphanumeric with hyphens
        "Customer Name": r".+",  # Any non-empty string
        "Contract Start Date": r"^\d{4}-\d{2}-\d{2}$",  # Date in YYYY-MM-DD
        "Contract End Date": r"^\d{4}-\d{2}-\d{2}$",  # Date in YYYY-MM-DD
        "Payment Terms": r"^[A-Za-z0-9\s\-]+$",  # Alphanumeric with spaces and hyphens
        "Contract Amount": r"^\$\d{1,3}(,\d{3})*(\.\d{2})?$",  # Currency format
        "Billing Frequency": r"^[A-Za-z0-9\s\-]+$",  # Alphanumeric with spaces and hyphens
        "Contract Type": r"^[A-Za-z0-9\s\-]+$"  # Alphanumeric with spaces and hyphens
    }

    # Refine fields
    refined_fields = {}
    for field, value in extracted_fields.items():
        if value is None or value == "Missing":  # Handle null or missing values
            refined_fields[field] = "Missing"
        elif field in patterns and not re.match(patterns[field], value, re.IGNORECASE):
            refined_fields[field] = "Invalid Format"
        else:
            refined_fields[field] = value  # Keep valid value as is

    return refined_fields

# Generate the dynamic prompt with reasoning
prompt_with_reasoning = create_dynamic_prompt_with_reasoning(fields, contract_text)

# Generate LLM output from Gemini
llm_output_with_reasoning = generate_llm_output_with_reasoning(prompt_with_reasoning)

# Refine fields without altering reasoning log
refined_fields = refine_fields_with_regex_only(llm_output_with_reasoning)

# Extract reasoning log as is
reasoning_log = llm_output_with_reasoning.get("reasoning_log", {})

# Print the refined fields
print("Refined Fields:", json.dumps(refined_fields, indent=4))

# Print the reasoning log
print("Reasoning Log:", json.dumps(reasoning_log, indent=4))


Refined Fields: {
    "Contract ID": "Missing",
    "Customer Name": "Company B",
    "Contract Start Date": "Invalid Format",
    "Contract End Date": "Missing",
    "Payment Terms": "Net 30",
    "Contract Amount": "$6,300",
    "Billing Frequency": "Monthly",
    "Contract Type": "Subscription"
}
Reasoning Log: {
    "Contract ID": "No specific Contract ID or Agreement Number could be identified in the document.",
    "Customer Name": "The 'Client' is identified as 'Company B' under the 'Client Information' section as well as  'Client Company Name' in the 'Order Form Acceptance and Authorization' section.",
    "Contract Start Date": "The contract start date is inferred from the signing date under the 'Order Form Acceptance and Authorization' section, which is '12/15/2023'.  The 'Effective Dates' section confirms the start date is tied to the signing date if 'Go Live' doesn't occur within a year.",
    "Contract End Date": "While the initial term is specified for Subscription Servic

#**3. Logging-Based Field Validation and Transformation**

This code validates and transforms extracted contract fields using default values for missing or invalid entries, logs detailed actions for transparency, and outputs the final validated fields in JSON format.

In [11]:
import logging
import json
import re
from io import StringIO

# Create a string buffer to capture logs
log_stream = StringIO()

# Custom log handler to write to StringIO
class StringIOHandler(logging.StreamHandler):
    def __init__(self, stream):
        super().__init__(stream)

# Configure logging
log_handler = StringIOHandler(log_stream)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
log_handler.setFormatter(formatter)

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(log_handler)  # Add custom handler for StringIO

def validate_and_transform_fields(refined_fields):
    """
    Validate the refined fields to ensure they meet the required formats.
    Log details of validation and transformations for transparency.
    """
    # Define default values for non-critical fields
    default_values = {
        "Contract ID": "Unknown",
        "Payment Terms": "Net 30",  # Default to "Net 30" if not provided
        "Contract Amount": "$0.00",  # Default to $0 if missing
        "Billing Frequency": "Monthly",  # Default to Monthly
        "Contract Type": "General Agreement"  # Default to a general agreement
    }

    # Validation results and explainable log
    validation_log = []

    for field, value in refined_fields.items():
        if value == "Missing" or value == "Invalid Format":
            # Log missing/invalid fields
            validation_log.append({
                "Field": field,
                "Status": "Missing or Invalid",
                "Action": f"Filling with default value: {default_values.get(field, 'None')}"
            })
            # Fill with default if non-critical
            refined_fields[field] = default_values.get(field, "None")
        else:
            # Perform format-specific validations
            if field in ["Contract Start Date", "Contract End Date"]:
                if not re.match(r"^\d{4}-\d{2}-\d{2}$", value):
                    validation_log.append({
                        "Field": field,
                        "Status": "Invalid Format",
                        "Action": "Flagging as Invalid"
                    })
                    refined_fields[field] = "Invalid Format"
                else:
                    validation_log.append({
                        "Field": field,
                        "Status": "Valid",
                        "Action": "No Action Needed"
                    })

            elif field == "Contract Amount":
                if not re.match(r"^\$\d{1,3}(,\d{3})*(\.\d{2})?$", value):
                    validation_log.append({
                        "Field": field,
                        "Status": "Invalid Format",
                        "Action": "Flagging as Invalid"
                    })
                    refined_fields[field] = "Invalid Format"
                else:
                    validation_log.append({
                        "Field": field,
                        "Status": "Valid",
                        "Action": "No Action Needed"
                    })

            else:
                # For all other fields, log as valid
                validation_log.append({
                    "Field": field,
                    "Status": "Valid",
                    "Action": "No Action Needed"
                })

    # Output the validation log for transparency
    for entry in validation_log:
        # Convert each dictionary entry to a JSON string for logging
        logger.info(json.dumps(entry, indent=4))

    return refined_fields

# Validate and Transform Fields
validated_and_transformed_fields = validate_and_transform_fields(refined_fields)

# Save logs to a string variable
logs_as_string = log_stream.getvalue()

# Print the final validated and transformed fields
# print("Final Validated and Transformed Fields:", json.dumps(validated_and_transformed_fields, indent=4))

# # Print the logs captured in the string variable
# print("\nCaptured Logs:\n", logs_as_string)


INFO:root:{
    "Field": "Contract ID",
    "Status": "Missing or Invalid",
    "Action": "Filling with default value: Unknown"
}
INFO:root:{
    "Field": "Customer Name",
    "Status": "Valid",
    "Action": "No Action Needed"
}
INFO:root:{
    "Field": "Contract Start Date",
    "Status": "Missing or Invalid",
    "Action": "Filling with default value: None"
}
INFO:root:{
    "Field": "Contract End Date",
    "Status": "Missing or Invalid",
    "Action": "Filling with default value: None"
}
INFO:root:{
    "Field": "Payment Terms",
    "Status": "Valid",
    "Action": "No Action Needed"
}
INFO:root:{
    "Field": "Contract Amount",
    "Status": "Valid",
    "Action": "No Action Needed"
}
INFO:root:{
    "Field": "Billing Frequency",
    "Status": "Valid",
    "Action": "No Action Needed"
}
INFO:root:{
    "Field": "Contract Type",
    "Status": "Valid",
    "Action": "No Action Needed"
}


Print the final fields and logs

In [12]:
# Print the final validated and transformed fields
print("Final Validated and Transformed Fields:", json.dumps(validated_and_transformed_fields, indent=4))

# # Print the logs captured in the string variable
print("\nCaptured Logs:\n", logs_as_string)

Final Validated and Transformed Fields: {
    "Contract ID": "Unknown",
    "Customer Name": "Company B",
    "Contract Start Date": "None",
    "Contract End Date": "None",
    "Payment Terms": "Net 30",
    "Contract Amount": "$6,300",
    "Billing Frequency": "Monthly",
    "Contract Type": "Subscription"
}

Captured Logs:
 2024-12-24 18:08:02,490 - INFO - {
    "Field": "Contract ID",
    "Status": "Missing or Invalid",
    "Action": "Filling with default value: Unknown"
}
2024-12-24 18:08:02,492 - INFO - {
    "Field": "Customer Name",
    "Status": "Valid",
    "Action": "No Action Needed"
}
2024-12-24 18:08:02,494 - INFO - {
    "Field": "Contract Start Date",
    "Status": "Missing or Invalid",
    "Action": "Filling with default value: None"
}
2024-12-24 18:08:02,497 - INFO - {
    "Field": "Contract End Date",
    "Status": "Missing or Invalid",
    "Action": "Filling with default value: None"
}
2024-12-24 18:08:02,500 - INFO - {
    "Field": "Payment Terms",
    "Status": "V

# **4. PDF Highlighting and Logging**

Highlights fields in a PDF using PyMuPDF and logs reasoning/confidence scores for explainability.

In [15]:
import fitz  # PyMuPDF for PDF parsing
import logging
import json
import re
from io import StringIO

# Create a string buffer to capture logs for explainability
explainability_log_stream = StringIO()

# Custom log handler to write to StringIO
class StringIOHandler(logging.StreamHandler):
    def __init__(self, stream):
        super().__init__(stream)

# Configure logging for explainability
log_handler = StringIOHandler(explainability_log_stream)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
log_handler.setFormatter(formatter)

logger = logging.getLogger("explainability_logger")
logger.setLevel(logging.INFO)
logger.addHandler(log_handler)  # Add custom handler for StringIO

# Step 1: Highlight PDF with Extracted Fields
def highlight_pdf(input_pdf_path, output_pdf_path, validated_and_transformed_fields):
    """
    Create a visual overlay on the PDF to highlight extracted fields.
    """
    try:
        # Open the original PDF
        pdf_document = fitz.open(input_pdf_path)

        for page_num, page in enumerate(pdf_document, start=1):
            for field, value in validated_and_transformed_fields.items():
                if value != "Missing" and value != "Invalid Format":
                    # Search for the text on the page
                    text_instances = page.search_for(value)
                    for inst in text_instances:
                        # Highlight the found text
                        page.add_highlight_annot(inst)

        # Save the modified PDF
        pdf_document.save(output_pdf_path, garbage=4, deflate=True)
        pdf_document.close()

        logger.info(f"PDF highlights saved to: {output_pdf_path}")

    except Exception as e:
        logger.error(f"Error highlighting PDF: {e}")

# Step 2: Log LLM Reasoning or Confidence
def log_llm_reasoning(validated_and_transformed_fields, reasoning_log):
    """
    Log LLM reasoning or confidence scores for extracted fields.
    """
    for field, reasoning in reasoning_log.items():
        log_entry = {
            "Field": field,
            "Validated Value": validated_and_transformed_fields.get(field, "Missing"),
            "Reasoning or Confidence": reasoning
        }
        logger.info(json.dumps(log_entry, indent=4))

# Integration with Validation and Transformation
# Example Input Data
validated_and_transformed_fields = {
    "Contract ID": "123-ABC",
    "Customer Name": "Reynolds Consumer Products Inc.",
    "Contract Start Date": "2020-02-04",
    "Contract End Date": "2021-02-04",
    "Payment Terms": "Net 30",
    "Contract Amount": "$10,000.00",
    "Billing Frequency": "Monthly",
    "Contract Type": "Transition Services Agreement"
}
reasoning_log = {
    "Contract ID": "Detected near the 'Agreement Number' label in the document.",
    "Customer Name": "Extracted from the 'Client' section of the PDF.",
    "Contract Start Date": "Identified using keywords like 'Effective Date'.",
    "Contract End Date": "Identified using keywords like 'Expiration Date'.",
    "Payment Terms": "Matched payment terms format like 'Net X'.",
    "Contract Amount": "Extracted near the 'Total Amount' section.",
    "Billing Frequency": "Detected as a recurring pattern in the document.",
    "Contract Type": "Matched with keywords like 'Agreement Type'."
}

# Highlight the PDF
input_pdf_path = "/content/sample contract.pdf"  # Path to your input PDF
output_pdf_path = "highlighted_contract.pdf"  # Path to save the highlighted PDF
highlight_pdf(input_pdf_path, output_pdf_path, validated_and_transformed_fields)

# Log LLM Reasoning
log_llm_reasoning(validated_and_transformed_fields, reasoning_log)

# Save explainability logs to a string variable
explainability_logs_as_string = explainability_log_stream.getvalue()

# Close the log stream
explainability_log_stream.close()

# Confirm Outputs
print(f"Highlighted PDF saved as: {output_pdf_path}")

INFO:explainability_logger:PDF highlights saved to: highlighted_contract.pdf
INFO:explainability_logger:{
    "Field": "Contract ID",
    "Validated Value": "123-ABC",
    "Reasoning or Confidence": "Detected near the 'Agreement Number' label in the document."
}
INFO:explainability_logger:{
    "Field": "Customer Name",
    "Validated Value": "Reynolds Consumer Products Inc.",
    "Reasoning or Confidence": "Extracted from the 'Client' section of the PDF."
}
INFO:explainability_logger:{
    "Field": "Contract Start Date",
    "Validated Value": "2020-02-04",
    "Reasoning or Confidence": "Identified using keywords like 'Effective Date'."
}
INFO:explainability_logger:{
    "Field": "Contract End Date",
    "Validated Value": "2021-02-04",
    "Reasoning or Confidence": "Identified using keywords like 'Expiration Date'."
}
INFO:explainability_logger:{
    "Field": "Payment Terms",
    "Validated Value": "Net 30",
    "Reasoning or Confidence": "Matched payment terms format like 'Net X'.

Highlighted PDF saved as: highlighted_contract.pdf

Explainability Logs:
 2024-12-24 18:17:54,033 - INFO - PDF highlights saved to: highlighted_contract.pdf
2024-12-24 18:17:54,035 - INFO - {
    "Field": "Contract ID",
    "Validated Value": "123-ABC",
    "Reasoning or Confidence": "Detected near the 'Agreement Number' label in the document."
}
2024-12-24 18:17:54,038 - INFO - {
    "Field": "Customer Name",
    "Validated Value": "Reynolds Consumer Products Inc.",
    "Reasoning or Confidence": "Extracted from the 'Client' section of the PDF."
}
2024-12-24 18:17:54,041 - INFO - {
    "Field": "Contract Start Date",
    "Validated Value": "2020-02-04",
    "Reasoning or Confidence": "Identified using keywords like 'Effective Date'."
}
2024-12-24 18:17:54,045 - INFO - {
    "Field": "Contract End Date",
    "Validated Value": "2021-02-04",
    "Reasoning or Confidence": "Identified using keywords like 'Expiration Date'."
}
2024-12-24 18:17:54,051 - INFO - {
    "Field": "Payment Terms

Printing Explainability/Highlighting Logs

In [16]:
print("\nExplainability Logs:\n", explainability_logs_as_string)


Explainability Logs:
 2024-12-24 18:17:54,033 - INFO - PDF highlights saved to: highlighted_contract.pdf
2024-12-24 18:17:54,035 - INFO - {
    "Field": "Contract ID",
    "Validated Value": "123-ABC",
    "Reasoning or Confidence": "Detected near the 'Agreement Number' label in the document."
}
2024-12-24 18:17:54,038 - INFO - {
    "Field": "Customer Name",
    "Validated Value": "Reynolds Consumer Products Inc.",
    "Reasoning or Confidence": "Extracted from the 'Client' section of the PDF."
}
2024-12-24 18:17:54,041 - INFO - {
    "Field": "Contract Start Date",
    "Validated Value": "2020-02-04",
    "Reasoning or Confidence": "Identified using keywords like 'Effective Date'."
}
2024-12-24 18:17:54,045 - INFO - {
    "Field": "Contract End Date",
    "Validated Value": "2021-02-04",
    "Reasoning or Confidence": "Identified using keywords like 'Expiration Date'."
}
2024-12-24 18:17:54,051 - INFO - {
    "Field": "Payment Terms",
    "Validated Value": "Net 30",
    "Reasoning 

#**5. Request to Zenskar API**

**POST** REQUEST

In [17]:
import requests
import json

# Define the Zenskar API endpoint
url = "https://api.zenskar.com/contract_v2"

# Example validated and transformed fields
validated_and_transformed_fields = {
    "Contract ID": "123-ABC",
    "Customer Name": "Reynolds Consumer Products Inc.",
    "Contract Start Date": "2024-12-24T12:25:21.032Z",
    "Contract End Date": "2024-12-24T12:25:21.032Z",
    "Payment Terms": "Net 30",
    "Contract Amount": "$10,000.00",
    "Billing Frequency": "Monthly",
    "Contract Type": "Subscription"
}

# Map the validated fields to the API's payload structure
payload = {
    "name": validated_and_transformed_fields.get("Customer Name", "Default Contract Name"),  # Contract name
    "description": f"Contract for {validated_and_transformed_fields.get('Customer Name', 'Unknown Customer')}",  # Contract description
    "status": "draft",  # Draft status
    "currency": "USD",  # Example currency
    "start_date": validated_and_transformed_fields.get("Contract Start Date"),
    "end_date": validated_and_transformed_fields.get("Contract End Date"),
    "customer_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",  # Example customer ID
    "anchor_date": validated_and_transformed_fields.get("Contract Start Date"),
    "plan_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",  # Example plan ID
    "phases": [
        {
            "name": "Initial Phase",
            "description": "Primary contract phase",
            "start_date": validated_and_transformed_fields.get("Contract Start Date"),
            "end_date": validated_and_transformed_fields.get("Contract End Date"),
            "pricings": [
                {
                    "external_id": "pricing_123",
                    "start_date": validated_and_transformed_fields.get("Contract Start Date"),
                    "end_date": validated_and_transformed_fields.get("Contract End Date"),
                    "pricing_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
                    "product_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6"
                }
            ],
            "features": {
                "name": "Standard Subscription",
                "description": "Basic subscription features",
                "pricing_data": {
                    "currency": "USD",
                    "label": "Subscription Pricing",
                    "unit": "month",
                    "pricing_period": {"cadence": "monthly"},
                    "unit_amount": 10000,  # Example value in cents
                    "dimensions": [{"name": "Feature Dimension", "column_name": "feature_dimension"}],
                    "prices": [10000],  # Example price in cents
                    "display_alias": ["Standard Plan"],
                    "pricing_type": "flat"
                },
                "quantity": {
                    "type": "fixed",
                    "label": "Fixed Quantity",
                    "quantity": 1,
                    "unit": "month",
                    "aggregate_id": "aggregate_123"
                },
                "is_recurring": True,
                "billing_period": {"cadence": "monthly", "offset": "prepaid"}
            },
            "source_plan_phase_id": "phase_123",
            "phase_type": "active"
        }
    ],
    "renewal_policy": "renew_with_default_contract",  # Example renewal policy
    "contract_link": "https://example.com/contracts/123-ABC"  # Example contract link
}

# Define headers
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

# Send the POST request
response = requests.post(url, json=payload, headers=headers)

# Print the response
print("Response Status Code:", response.status_code)
print("Response Body:", response.text)


Response Status Code: 403
Response Body: {"Message":"User is not authorized to access this resource with an explicit deny"}

